# Batch conversion Dicom to NRRD in XNAT collection

### import necessary components

In [1]:
#ORAW modifications and integrations with XNAT by Leonard Wee during Oct -> Dec 2021

from __future__ import print_function

from time import process_time
import os, subprocess, glob
#import yaml
#import ORAW
import glob
import shutil
import pandas as pd
from datetime import date, datetime
from numpy.random import choice

import xnat #needed for reading and writing to/from XNAT
from DicomDatabase import DicomDatabase # dicom indexer by Johan van Soest


### define needed operations

In [2]:
def upload_file(session, project, subject, experiment, assessment, resource, convfile):
    xnat_project = session.projects[project]
    xnat_subject = session.classes.SubjectData(parent=xnat_project, label=subject)
    xnat_experiment = session.classes.CtSessionData(parent=xnat_subject, label=experiment)
    xnat_resource = session.classes.ResourceCatalog(parent=xnat_experiment, label=resource)
    #xnat_assessment = session.classes.QcAssessmentData(parent=xnat_experiment, label=assessment) #not used
    #xnat_resource = session.classes.ResourceCatalog(parent=xnat_assessment, label=resource) #resource under experiment instead
    xnat_resource.upload(convfile, os.path.basename(convfile)) # upload
#    for file_ in data:
#        resource.upload(file_, os.path.basename(file_))
#    pass


def cleanup_temp_folder(dirt):
    if os.path.exists(dirt):
        shutil.rmtree(dirt)
        os.makedirs(dirt)
    if not os.path.exists(dirt):
        os.makedirs(dirt)
        
def RTSTRUCT2NRRD(pt,dt):
    # convert contents of STRUCTWorkingDir to NRRD using CTWorkingDir as reference
    f = open(os.path.join('./plastimatch','plastimatch_logfile.txt'), "a")
    try:
        subprocess.run(['plastimatch', 'convert',\
                '--input',STRUCTWorkingDir,\
                '--output-prefix',NRRDWorkingDir,
                '--prefix-format','nrrd',\
                '--referenced-ct',CTWorkingDir,\
                '--prune-empty'], stdout=f)
    except:
        print("Error: plastimatch failed to convert RTSTRUCT to [roiname].nrrd", file=f)
    print("Subject: %s" % (pt), file=f)
    print("Datetime stamp : %s" % (dt), file=f)
    f.close()

def AXIALCT2NRRD(pt,dt):
    # convert contents of CTWorkingDir to NRRD
    f = open(os.path.join('./plastimatch','plastimatch_logfile.txt'), "a")
    try:
        subprocess.run(['plastimatch', 'convert',\
              '--input',CTWorkingDir,\
              '--output-img',os.path.join(NRRDWorkingDir,'ct_image.nrrd')], stdout=f)
    except:
        print("Error: plastimatch failed to convert DICOM CT image to ct_image.nrrd", file=f)
    print("Subject: %s" % (pt), file=f)
    print("Datetime stamp : %s" % (dt), file=f)
    f.close()


### configure user-dependent settings

In [4]:
#------------------------- USER SETTINGS -------------------------------

# set up XNAT login credentials here
# .......................
#xnatUrl = 'https://xnat.bmia.nl'
#xnatUser = 'leonardwee'
xnatUrl = 'http://172.24.0.5:80'  #change me! e.g. 'http://localhost:8081/'
xnatUser = 'admin'  #change me!
xnatPass = 'admin'  #change me!
xnatProject = 'hn3-radiomics'  #change me!

### retrieve list of patients and dicom experiments from XNAT project

In [5]:
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            print(xnatProject + "\t" + mySubjectID + "\t" + myExperimentID)

[WARNING] Detected a redirect from http://172.24.0.5:80 to http://172.24.0.5/, using http://172.24.0.5/ from now on


hn3-radiomics	P0415C0006I355015	P0415C0006I355015
hn3-radiomics	P0415C0006I1454678	P0415C0006I1454678
hn3-radiomics	P0415C0006I1263056	P0415C0006I1263056
hn3-radiomics	P0415C0006I1660348	P0415C0006I1660348
hn3-radiomics	P0415C0006I1447565	P0415C0006I1447565
hn3-radiomics	P0415C0006I119663	P0415C0006I119663
hn3-radiomics	P0415C0006I1097833	P0415C0006I1097833
hn3-radiomics	P0415C0006I1439383	P0415C0006I1439383
hn3-radiomics	P0415C0006I13300	P0415C0006I13300
hn3-radiomics	P0415C0006I1146494	P0415C0006I1146494
hn3-radiomics	P0415C0006I2024685	P0415C0006I2024685
hn3-radiomics	P0415C0006I1733057	P0415C0006I1733057
hn3-radiomics	P0415C0006I1966784	P0415C0006I1966784
hn3-radiomics	P0415C0006I1483114	P0415C0006I1483114
hn3-radiomics	P0415C0006I1513638	P0415C0006I1513638
hn3-radiomics	P0415C0006I1515168	P0415C0006I1515168
hn3-radiomics	P0415C0006I1827573	P0415C0006I1827573
hn3-radiomics	P0415C0006I2045556	P0415C0006I2045556
hn3-radiomics	P0415C0006I1145536	P0415C0006I1145536
hn3-radiomics	P0415C

### main section

In [8]:
XNATdownload = "./XnatDownload"
CTWorkingDir = "./CTFolder"
STRUCTWorkingDir = "./StructFolder"
NRRDWorkingDir = "./NrrdFolder"
currentResourceLabel = datetime.now().strftime("%Y%m%d_%H%M")
#
#----------------O-RAW pyradiomics parameters -------------------
pyradiomics_params_file = './ParamsSettings/Pyradiomics_Params.yaml' #change this to your own params if needed


start_time = process_time()

# -----------------------------------------------------------
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            cleanup_temp_folder(XNATdownload)
            myExperiment.download_dir(XNATdownload)

            # initialize dicom DB
            dicomDb = DicomDatabase()
            dicomDb.parseFolder(XNATdownload)
        
            # main
            for ptid in dicomDb.getPatientIds():
                print("Processing: %s" % (ptid)) # get patient by ID
                myChoice = choice(['D','V'],p=[0.8,0.2]) #selects the patient into development subset with prob 0.8
                
                myPatient = dicomDb.getPatient(ptid)
                # loop over RTStructs of this patient
                for myStructUID in myPatient.getRTStructs():
                    print("Starting with RTStruct %s" % myStructUID)
                    # Get RTSTRUCT by SOP Instance UID
                    myStruct = myPatient.getRTStruct(myStructUID)
                    # Get CT which is referenced by this RTStruct, and is linked to the same patient
                    # mind that this can be None, as only a struct, without corresponding CT scan is found
                    myCT = myPatient.getCTForRTStruct(myStruct)
                
                    # clear the working CT/STRUCT folder
                    cleanup_temp_folder(CTWorkingDir)
                    cleanup_temp_folder(STRUCTWorkingDir)
                        
                    #only if we have both RTStruct and CT
                    if myCT is not None:
                        shutil.move(myStruct.getFileLocation(),os.path.join(STRUCTWorkingDir,'struct.dcm')) # move RTSTRUCT file to tmp folder as 'struct.dcm'
                        slices = myCT.getSlices()
                        for i in range(len(slices)):
                            shutil.move(slices[i],os.path.join(CTWorkingDir,str(i)+".dcm")) #move CT files to tmp folder as incremental file names
                        
                        right_now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
                        cleanup_temp_folder(NRRDWorkingDir)
                        RTSTRUCT2NRRD(ptid,right_now)
                        AXIALCT2NRRD(ptid,right_now)
                    
                    
                    #filter the produced NRRD files for GTV-[0-9] and ct_image
                    for fname in glob.glob(NRRDWorkingDir + "/GTV*.nrrd"):
                        print("Located GTV file : "+fname)
                        resource_folder = myChoice + '_' + currentResourceLabel
                        resource_folder = resource_folder + '_' + fname.split('/')[-1].split(".")[0]
                        #resource_folder = datetime.now().strftime("%Y%m%d_%H%M")
                        try:
                            #upload_file(session, xnatProject, mySubjectID, myExperimentID, '0', resource_folder, fname)
                            print("Saved MASK as resource in XNAT under experiment.")
                            print("%s;%s;%s;%s;%s" %(xnatProject,mySubjectID, myExperimentID, resource_folder, fname))
                        except:
                            print("Failed to save MASK to XNAT!")                            
                    
                    #NRRD files for CT image - assumed to be only one at a time in NRRDWorkingDir
                    for fname in glob.glob(NRRDWorkingDir + "/ct_image.nrrd"):
                        print("Located CT IMAGE file : "+fname)
                        #resource_folder = fname.split('/')[-1].split(".")[0]
                        resource_folder = myChoice + '_' + currentResourceLabel
                        resource_folder = resource_folder + "_ct_image"
                        try:
                            #upload_file(session, xnatProject, mySubjectID, myExperimentID, '0', resource_folder, fname)
                            print("Saved CT_IMAGE as resource in XNAT under experiment.")
                            print("%s;%s;%s;%s;%s" %(xnatProject,mySubjectID, myExperimentID, resource_folder, fname))
                        except:
                            print("Failed to save CT_IMAGE to XNAT!")
                    
                    #pyradiomics values possibly useful for comparing GTVs
                    #-- comment out the following to skip over
                    for fname in glob.glob(NRRDWorkingDir + "/GTV*.nrrd"):
                        ctname = os.path.join(NRRDWorkingDir,'ct_image.nrrd') #assume fixed
                        rname = fname.split('/')[-1].split(".")[0] + '.csv' #named as GTV-[0-9].csv
                        rname = os.path.join(NRRDWorkingDir,rname) #fixes small error with output to csv file
                        resource_folder = "pyr_" + currentResourceLabel
                        resource_folder = resource_folder + '_' + fname.split('/')[-1].split(".")[0]
                        #run pyradiomics
                        subprocess.run(['pyradiomics',ctname, fname,\
                                            '--param',pyradiomics_params_file,\
                                            '-o',rname,'-f','csv'],capture_output=True)
                        try:
                            upload_file(session, xnatProject, mySubjectID, myExperimentID, '0', resource_folder, rname)
                        except:
                            print("--- error processing radiomics! -----")
                    
                    #conclude and move onto next subject
                    print("Done for RTStruct %s of subject %s .... moving on ...." % (myStructUID, ptid))
                    
                    #only use the break for the purpose of testing
                    #break

##### ----------------------------------------------------------------------------

stop_time = process_time()
print("Time elapsed : %s" % (stop_time-start_time))


[WARNING] Detected a redirect from http://172.24.0.5:80 to http://172.24.0.5/, using http://172.24.0.5/ from now on
119.3 MiB |                #                                      |   1.5 MiB/s


Processing: P0415C0006I355015
Starting with RTStruct 1.3.6.1.4.1.32722.415.6.334438723428200308149604515333735375401
Located GTV file : ./NrrdFolder/GTVp1.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I355015;P0415C0006I355015;D_20220104_2357_GTVp1;./NrrdFolder/GTVp1.nrrd
Located CT IMAGE file : ./NrrdFolder/ct_image.nrrd
Saved CT_IMAGE as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I355015;P0415C0006I355015;D_20220104_2357_ct_image;./NrrdFolder/ct_image.nrrd
Done for RTStruct 1.3.6.1.4.1.32722.415.6.334438723428200308149604515333735375401 of subject P0415C0006I355015 .... moving on ....


 91.8 MiB |                                #                      |   1.4 MiB/s


Processing: P0415C0006I1454678
Starting with RTStruct 1.3.6.1.4.1.32722.415.6.187407374563272489748043799716277894637
Located GTV file : ./NrrdFolder/GTV_1.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I1454678;P0415C0006I1454678;D_20220104_2357_GTV_1;./NrrdFolder/GTV_1.nrrd
Located GTV file : ./NrrdFolder/GTV_2.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I1454678;P0415C0006I1454678;D_20220104_2357_GTV_2;./NrrdFolder/GTV_2.nrrd
Located CT IMAGE file : ./NrrdFolder/ct_image.nrrd
Saved CT_IMAGE as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I1454678;P0415C0006I1454678;D_20220104_2357_ct_image;./NrrdFolder/ct_image.nrrd
Done for RTStruct 1.3.6.1.4.1.32722.415.6.187407374563272489748043799716277894637 of subject P0415C0006I1454678 .... moving on ....


 75.4 MiB |                               #                       |   1.3 MiB/s


...

Processing: P0415C0006I9769068
Starting with RTStruct 1.3.6.1.4.1.32722.415.6.102577419828671826464155128533592054162
Located GTV file : ./NrrdFolder/GTV-1.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9769068;P0415C0006I9769068;D_20220104_2357_GTV-1;./NrrdFolder/GTV-1.nrrd
Located GTV file : ./NrrdFolder/GTV-2.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9769068;P0415C0006I9769068;D_20220104_2357_GTV-2;./NrrdFolder/GTV-2.nrrd
Located CT IMAGE file : ./NrrdFolder/ct_image.nrrd
Saved CT_IMAGE as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9769068;P0415C0006I9769068;D_20220104_2357_ct_image;./NrrdFolder/ct_image.nrrd
Done for RTStruct 1.3.6.1.4.1.32722.415.6.102577419828671826464155128533592054162 of subject P0415C0006I9769068 .... moving on ....


101.0 MiB |                                                   #   |   1.5 MiB/s


Processing: P0415C0006I9565578
Starting with RTStruct 1.3.6.1.4.1.32722.415.6.292671665377919749419311355994243456136
Located GTV file : ./NrrdFolder/GTV-1.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9565578;P0415C0006I9565578;V_20220104_2357_GTV-1;./NrrdFolder/GTV-1.nrrd
Located GTV file : ./NrrdFolder/GTV-2preop.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9565578;P0415C0006I9565578;V_20220104_2357_GTV-2preop;./NrrdFolder/GTV-2preop.nrrd
Located GTV file : ./NrrdFolder/GTV-3.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9565578;P0415C0006I9565578;V_20220104_2357_GTV-3;./NrrdFolder/GTV-3.nrrd
Located GTV file : ./NrrdFolder/GTVPTV.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9565578;P0415C0006I9565578;V_20220104_2357_GTVPTV;./NrrdFolder/GTVPTV.nrrd
Located CT IMAGE file : ./NrrdFolder/ct_image.nrrd
Saved CT_IMAGE as resource in XNAT under experiment.


 98.9 MiB |               #                                       |   1.3 MiB/s


Processing: P0415C0006I9888200
Starting with RTStruct 1.3.6.1.4.1.32722.415.6.158813174999552320433001884980950455128
Located GTV file : ./NrrdFolder/GTVn_L.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9888200;P0415C0006I9888200;D_20220104_2357_GTVn_L;./NrrdFolder/GTVn_L.nrrd
Located GTV file : ./NrrdFolder/GTVp1.nrrd
Saved MASK as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9888200;P0415C0006I9888200;D_20220104_2357_GTVp1;./NrrdFolder/GTVp1.nrrd
Located CT IMAGE file : ./NrrdFolder/ct_image.nrrd
Saved CT_IMAGE as resource in XNAT under experiment.
hn3-radiomics;P0415C0006I9888200;P0415C0006I9888200;D_20220104_2357_ct_image;./NrrdFolder/ct_image.nrrd
Done for RTStruct 1.3.6.1.4.1.32722.415.6.158813174999552320433001884980950455128 of subject P0415C0006I9888200 .... moving on ....
Time elapsed : 1053.7757803999998
